In [330]:
import pandas as pd
from create_advancement_sheets import AdvancementUtils
import pandas.io.formats.excel
utils = AdvancementUtils()


pandas.io.formats.excel.header_style = None

In [331]:
df = utils.read_sheet("data.xlsx", True)
df = utils.pivot_names_horizontal(df)

In [332]:
bats = df.loc[:, ("Bateleurs")]
bats = bats.reset_index()

In [333]:
bats.Level.cat.categories

Index(['Membership', 'Traveller', 'Discoverer', '1st Class', 'Springbok'], dtype='object')

In [334]:
len(bats.loc[(bats.Level == "Traveller") & (bats.Theme == "Safety"), :])

7

In [335]:
levels = ["Membership", "Traveller", "Discoverer", "1st Class", "Springbok"]
themes = ["Safety", "Living Outdoors", "Adventure", "Scout Skills", "Service", "Personal Dev"]

In [336]:
writer = pd.ExcelWriter("test.xlsx", engine='xlsxwriter')

In [337]:
first_row_df = 1
bats.to_excel(writer, sheet_name = "Bats", index=False, startrow=first_row_df)

In [338]:
workbook = writer.book
worksheet = writer.sheets["Bats"]


In [339]:
worksheet.set_portrait()
worksheet.set_paper(8)
worksheet.set_margins(0.25, 0.25, 0.25, 0.25)

In [340]:
level_format = workbook.add_format({"bold": True, "font_size": 14, "font_name": "Verdana", "align": "center", "valign":"vcenter", "rotation":90})
theme_format = workbook.add_format({"bold": True, "font_size": 12, "font_name": "Verdana", "align": "center", "valign":"vcenter", "rotation":90, "text_wrap": True})
requirement_fmt = workbook.add_format({"font_size": 10, "font_name": "Verdana", "align": "horizontal", "valign":"vcenter"})
true_format = workbook.add_format({"font_color": "black", "bg_color": "black"})
false_format = workbook.add_format({"font_color": "white", "bg_color": "white"})
header_format = workbook.add_format({"bold": True, "font_size": 9, "font_name": "Verdana", "align": "center", "valign":"vcenter", "text_wrap": True})
border_format = workbook.add_format({"border": 1})
title_format = workbook.add_format({"bold": True, "font_size": 20, "font_name": "Verdana", "align": "center", "valign":"vcenter"})

In [341]:
## Merging levels

start_row = first_row_df + 2
for level in levels:
    end_row = start_row + len(bats.loc[bats.Level == level, :])
    worksheet.merge_range("A{}:A{}".format(start_row, end_row-1), level, level_format)
    start_row = end_row

In [342]:
## Merging themes
worksheet.merge_range("B{}:B{}".format(first_row_df+2, first_row_df+7), "", theme_format)
start_row =  first_row_df + 8 # inclusive
for level in levels[1:]:
    for theme in themes:
        end_row = start_row + len(bats.loc[(bats.Level == level) & (bats.Theme == theme), :])
        if end_row - start_row  > 1:
            worksheet.merge_range("B{}:B{}".format(start_row, end_row-1), theme, theme_format)
        else:
            worksheet.write("B{}".format(start_row), theme, theme_format)
        start_row = end_row

In [343]:
worksheet.set_column('A:A', 5)
worksheet.set_column('B:B', 5)
worksheet.set_column('C:C', 30, requirement_fmt)

0

In [344]:
full_data_range = "A{}:{}{}".format(first_row_df+1, "J", first_row_df+134)
pass_range = "D{}:{}{}".format(first_row_df+2, "J", first_row_df+134)

worksheet.conditional_format(pass_range, {'type':     'cell',
                                        'criteria': '=',
                                        'value':    True,
                                        'format':   true_format})

worksheet.conditional_format(pass_range, {'type':     'cell',
                                        'criteria': '=',
                                        'value':    False,
                                        'format':   false_format})

worksheet.conditional_format(full_data_range, {"type": "no_blanks", "format": border_format})                                        

0

In [345]:
worksheet.set_column(4, len(bats.columns.to_list())-4, 90//7)

0

In [346]:

for i in range(first_row_df+1, 135):
    worksheet.set_row_pixels(i, 25)

In [347]:
header_to_write = bats.columns.to_list()
header_to_write[0] = " "
header_to_write[1] = " "

worksheet.write_row(first_row_df, 0, header_to_write)
worksheet.set_row_pixels(first_row_df, 40, header_format)

0

In [348]:
worksheet.set_row_pixels(0, 40)
worksheet.merge_range("A1:J1", "Bats Advancement", title_format)

0

In [349]:
writer.close()